In [143]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.chat_models import ChatOllama
import pandas as pd
import json

## 提取出文本中的物品

In [144]:
# Temp = 0 so that we get clean information without a lot of creativity
chat_model = ChatOllama(temperature=0, model="qwen:32b-chat-v1.5-q5_K_M")

text_input = """
林克推开门进入卧室，房间布置简洁而温馨。一张坚固的木床靠在北墙正中，床头上方挂着一幅描绘着塞尔达大陆的装饰画，床尾朝向南边的窗户，让林克在清晨醒来时能第一时间感受到阳光的温暖。
床的右侧是一个小巧的书桌，书桌面向东边，林克可以在这里规划他的冒险路线，书桌上整齐地摆放着地图和羽毛笔。在房间的西南角，一个高挑的衣柜紧贴墙壁，柜门上刻有海拉尔的纹章，内部整齐地挂着林克的服装和装备。
衣柜对面，一张舒适的靠背椅放置在东墙边，旁边是一张小茶几，上面常放着一杯热茶，供林克在休息时享用。整个房间的布局以方便和实用为主，同时不失为一个放松和规划冒险的理想空间。
"""

get_object_chain = ChatPromptTemplate.from_template("请提取出如下文本中的陈设和物品，输出一个数组。文本：{text_input}") | chat_model | StrOutputParser()

objects_in_text = get_object_chain.invoke({"text_input": text_input})

print(objects_in_text)

['木床', '装饰画', '书桌', '地图', '羽毛笔', '衣柜', '海拉尔的纹章', '服装', '装备', '靠背椅', '小茶几', '热茶']


## 在素材库中找到匹配的物品

In [145]:
# How you would like your response structured. This is basically a fancy prompt template
response_schemas = [
    ResponseSchema(name="object_in_text", description="这是用户输入的物品"),
    ResponseSchema(name="object_in_gallery", description="这是你觉得与用户输入最匹配的物品"),
    ResponseSchema(name="match_score", description="你认为用户输入的匹配与你匹配的接近程度的0-100分")
]

# How you would like to parse your output
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# See the prompt template you created for formatting
format_instructions = output_parser.get_format_instructions()
print (output_parser.get_format_instructions())

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"object_in_text": string  // 这是用户输入的物品
	"object_in_gallery": string  // 这是你觉得与用户输入最匹配的物品
	"match_score": string  // 你认为用户输入的匹配与你匹配的接近程度的0-100分
}
```


In [146]:
template = """
您将获得用户提供的一系列物品名称。
在已入库物品列表中找到最合适的匹配项。
最接近的匹配将是具有最接近语义的匹配。不仅仅是字符串相似。

{format_instructions}

用左括号和右括号包装你的最终输出(一个json对象列表)

用户输入的物品:
{objects_in_text}

已入库物品:
{objects_in_gallery}

你的反应:
"""

prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(template)  
    ],
    input_variables=["object_in_text", "objects_in_gallery"],
    partial_variables={"format_instructions": format_instructions}
)

# Get your standardized names. You can swap this out with whatever list you want!
df = pd.read_csv('../data/ObjectsInGallery.csv')
objects_in_gallery = ", ".join(df['物品'].values)

# Your user input
_input = prompt.format_prompt(objects_in_text=objects_in_text, objects_in_gallery=objects_in_gallery)

print (f"There are {len(_input.messages)} message(s)")
print (f"Type: {type(_input.messages[0])}")
print ("---------------------------")
print (_input.messages[0].content)

output = chat_model(_input.to_messages())

There are 1 message(s)
Type: <class 'langchain_core.messages.human.HumanMessage'>
---------------------------

您将获得用户提供的一系列物品名称。
在已入库物品列表中找到最合适的匹配项。
最接近的匹配将是具有最接近语义的匹配。不仅仅是字符串相似。

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"object_in_text": string  // 这是用户输入的物品
	"object_in_gallery": string  // 这是你觉得与用户输入最匹配的物品
	"match_score": string  // 你认为用户输入的匹配与你匹配的接近程度的0-100分
}
```

用左括号和右括号包装你的最终输出(一个json对象列表)

用户输入的物品:
['木床', '装饰画', '书桌', '地图', '羽毛笔', '衣柜', '海拉尔的纹章', '服装', '装备', '靠背椅', '小茶几', '热茶']

已入库物品:
床1, 床2, 床3, 书桌1, 书桌2, 书桌3, 衣柜1, 衣柜2, 衣柜3, 靠背椅1, 靠背椅2, 靠背椅3

你的反应:



In [147]:
if "```json" in output.content:
    json_string = output.content.split("```json")[1].strip()
else:
    json_string = output.content

if "```" in json_string:
    json_string = json_string.split("```")[0].strip()

print(json_string)

[
  {
    "object_in_text": "木床",
    "object_in_gallery": "床1",
    "match_score": "90"
  },
  {
    "object_in_text": "装饰画",
    "object_in_gallery": "无匹配项",
    "match_score": "0"
  },
  {
    "object_in_text": "书桌",
    "object_in_gallery": "书桌1",
    "match_score": "90"
  },
  {
    "object_in_text": "地图",
    "object_in_gallery": "无匹配项",
    "match_score": "0"
  },
  {
    "object_in_text": "羽毛笔",
    "object_in_gallery": "无匹配项",
    "match_score": "0"
  },
  {
    "object_in_text": "衣柜",
    "object_in_gallery": "衣柜1",
    "match_score": "90"
  },
  {
    "object_in_text": "海拉尔的纹章",
    "object_in_gallery": "无匹配项",
    "match_score": "0"
  },
  {
    "object_in_text": "服装",
    "object_in_gallery": "无匹配项",
    "match_score": "0"
  },
  {
    "object_in_text": "装备",
    "object_in_gallery": "无匹配项",
    "match_score": "0"
  },
  {
    "object_in_text": "靠背椅",
    "object_in_gallery": "靠背椅1",
    "match_score": "90"
  },
  {
    "object_in_text": "小茶几",
    "object_in_gallery": "无匹

In [148]:
# output_parser.parse(output.content) Ideally this works but not in all cases
structured_data = json.loads(json_string)
structured_data

[{'object_in_text': '木床', 'object_in_gallery': '床1', 'match_score': '90'},
 {'object_in_text': '装饰画', 'object_in_gallery': '无匹配项', 'match_score': '0'},
 {'object_in_text': '书桌', 'object_in_gallery': '书桌1', 'match_score': '90'},
 {'object_in_text': '地图', 'object_in_gallery': '无匹配项', 'match_score': '0'},
 {'object_in_text': '羽毛笔', 'object_in_gallery': '无匹配项', 'match_score': '0'},
 {'object_in_text': '衣柜', 'object_in_gallery': '衣柜1', 'match_score': '90'},
 {'object_in_text': '海拉尔的纹章', 'object_in_gallery': '无匹配项', 'match_score': '0'},
 {'object_in_text': '服装', 'object_in_gallery': '无匹配项', 'match_score': '0'},
 {'object_in_text': '装备', 'object_in_gallery': '无匹配项', 'match_score': '0'},
 {'object_in_text': '靠背椅', 'object_in_gallery': '靠背椅1', 'match_score': '90'},
 {'object_in_text': '小茶几', 'object_in_gallery': '无匹配项', 'match_score': '0'},
 {'object_in_text': '热茶', 'object_in_gallery': '无匹配项', 'match_score': '0'}]

In [149]:
pd.DataFrame(structured_data)

,object_in_text,object_in_gallery,match_score
0,木床,床1,90
1,装饰画,无匹配项,0
2,书桌,书桌1,90
3,地图,无匹配项,0
4,羽毛笔,无匹配项,0
5,衣柜,衣柜1,90
6,海拉尔的纹章,无匹配项,0
7,服装,无匹配项,0
8,装备,无匹配项,0
9,靠背椅,靠背椅1,90


## 根据文本确定物品方位和坐标

In [150]:
# How you would like your response structured. This is basically a fancy prompt template
response_schemas = [
    ResponseSchema(name="object_in_text", description="这是用户输入的物品"),
    ResponseSchema(name="place", description="这是根据上下文物品的位置"),
    ResponseSchema(name="direction", description="这是根据上下文物品的朝向（东、西、南、北、上、下）"),
    ResponseSchema(name="translation_x", description="这是物品的位置X坐标"),
    ResponseSchema(name="translation_y", description="这是物品的位置Y坐标"),
    ResponseSchema(name="translation_z", description="这是物品的位置Z坐标"),
    ResponseSchema(name="rotation_x", description="这是物品的X旋转角"),
    ResponseSchema(name="rotation_y", description="这是物品的Y旋转角"),
    ResponseSchema(name="rotation_z", description="这是物品的Z旋转角")
]

# How you would like to parse your output
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# See the prompt template you created for formatting
format_instructions = output_parser.get_format_instructions()
print (output_parser.get_format_instructions())

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"object_in_text": string  // 这是用户输入的物品
	"place": string  // 这是根据上下文物品的位置
	"direction": string  // 这是根据上下文物品的朝向（东、西、南、北、上、下）
	"translation_x": string  // 这是物品的位置X坐标
	"translation_y": string  // 这是物品的位置Y坐标
	"translation_z": string  // 这是物品的位置Z坐标
	"rotation_x": string  // 这是物品的X旋转角
	"rotation_y": string  // 这是物品的Y旋转角
	"rotation_z": string  // 这是物品的Z旋转角
}
```


In [151]:
template = """
您将获得用户提供的一系列物品名称。
根据上下文推断出物品的位置和朝向。
重点考虑物品之间的相对位置关系，位置应符合上下文的描述，同时避免重叠和碰撞。
如果有描述边界，物品不要超出边界。
X轴指向东、Y轴指向北、Z轴指向上。
物体旋转角为欧拉旋转角

{format_instructions}

用左括号和右括号包装你的最终输出(一个json对象列表)

用户输入的物品:
{objects_in_text}

上下文:
{text_input}

你的反应:
"""

prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(template)  
    ],
    input_variables=["object_in_text", "text_input"],
    partial_variables={"format_instructions": format_instructions}
)

# Your user input
_input = prompt.format_prompt(objects_in_text=objects_in_text, text_input=text_input)


print (f"There are {len(_input.messages)} message(s)")
print (f"Type: {type(_input.messages[0])}")
print ("---------------------------")
print (_input.messages[0].content)

output = chat_model(_input.to_messages())

There are 1 message(s)
Type: <class 'langchain_core.messages.human.HumanMessage'>
---------------------------

您将获得用户提供的一系列物品名称。
根据上下文推断出物品的位置和朝向。
重点考虑物品之间的相对位置关系，位置应符合上下文的描述，同时避免重叠和碰撞。
如果有描述边界，物品不要超出边界。
X轴指向东、Y轴指向北、Z轴指向上。
物体旋转角为欧拉旋转角

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"object_in_text": string  // 这是用户输入的物品
	"place": string  // 这是根据上下文物品的位置
	"direction": string  // 这是根据上下文物品的朝向（东、西、南、北、上、下）
	"translation_x": string  // 这是物品的位置X坐标
	"translation_y": string  // 这是物品的位置Y坐标
	"translation_z": string  // 这是物品的位置Z坐标
	"rotation_x": string  // 这是物品的X旋转角
	"rotation_y": string  // 这是物品的Y旋转角
	"rotation_z": string  // 这是物品的Z旋转角
}
```

用左括号和右括号包装你的最终输出(一个json对象列表)

用户输入的物品:
['木床', '装饰画', '书桌', '地图', '羽毛笔', '衣柜', '海拉尔的纹章', '服装', '装备', '靠背椅', '小茶几', '热茶']

上下文:

林克推开门进入卧室，房间布置简洁而温馨。一张坚固的木床靠在北墙正中，床头上方挂着一幅描绘着塞尔达大陆的装饰画，床尾朝向南边的窗户，让林克在清晨醒来时能第一时间感受到阳光的温暖。
床的右侧是一个小巧的书桌，书桌面向东边，林克可以在这里规划他的冒险路线，书桌上整齐

In [152]:
if "```json" in output.content:
    json_string = output.content.split("```json")[1].strip()
else:
    json_string = output.content

if "```" in json_string:
    json_string = json_string.split("```")[0].strip()

print(json_string)

[
  {
    "object_in_text": "木床",
    "place": "北墙正中",
    "direction": "南北向",
    "translation_x": "0",
    "translation_y": "-2",
    "translation_z": "0",
    "rotation_x": "0",
    "rotation_y": "90",
    "rotation_z": "0"
  },
  {
    "object_in_text": "装饰画",
    "place": "床头墙上",
    "direction": "南北向",
    "translation_x": "0",
    "translation_y": "-1.5",
    "translation_z": "0",
    "rotation_x": "0",
    "rotation_y": "90",
    "rotation_z": "0"
  },
  {
    "object_in_text": "书桌",
    "place": "床右侧，面向东边",
    "direction": "东西向",
    "translation_x": "1",
    "translation_y": "-2",
    "translation_z": "0",
    "rotation_x": "90",
    "rotation_y": "0",
    "rotation_z": "0"
  },
  {
    "object_in_text": "地图",
    "place": "书桌上",
    "direction": "",
    "translation_x": "1.5",
    "translation_y": "-2.5",
    "translation_z": "0",
    "rotation_x": "0",
    "rotation_y": "0",
    "rotation_z": "0"
  },
  {
    "object_in_text": "羽毛笔",
    "place": "书桌上",
    "direction": ""

In [153]:
# output_parser.parse(output.content) Ideally this works but not in all cases
structured_data = json.loads(json_string)
structured_data

[{'object_in_text': '木床',
  'place': '北墙正中',
  'direction': '南北向',
  'translation_x': '0',
  'translation_y': '-2',
  'translation_z': '0',
  'rotation_x': '0',
  'rotation_y': '90',
  'rotation_z': '0'},
 {'object_in_text': '装饰画',
  'place': '床头墙上',
  'direction': '南北向',
  'translation_x': '0',
  'translation_y': '-1.5',
  'translation_z': '0',
  'rotation_x': '0',
  'rotation_y': '90',
  'rotation_z': '0'},
 {'object_in_text': '书桌',
  'place': '床右侧，面向东边',
  'direction': '东西向',
  'translation_x': '1',
  'translation_y': '-2',
  'translation_z': '0',
  'rotation_x': '90',
  'rotation_y': '0',
  'rotation_z': '0'},
 {'object_in_text': '地图',
  'place': '书桌上',
  'direction': '',
  'translation_x': '1.5',
  'translation_y': '-2.5',
  'translation_z': '0',
  'rotation_x': '0',
  'rotation_y': '0',
  'rotation_z': '0'},
 {'object_in_text': '羽毛笔',
  'place': '书桌上',
  'direction': '',
  'translation_x': '1.2',
  'translation_y': '-2.5',
  'translation_z': '0',
  'rotation_x': '0',
  'rotation_

In [154]:
pd.DataFrame(structured_data)

,object_in_text,place,direction,translation_x,translation_y,translation_z,rotation_x,rotation_y,rotation_z
0,木床,北墙正中,南北向,0,-2,0,0,90,0
1,装饰画,床头墙上,南北向,0,-1.5,0,0,90,0
2,书桌,床右侧，面向东边,东西向,1,-2,0,90,0,0
3,地图,书桌上,,1.5,-2.5,0,0,0,0
4,羽毛笔,书桌上,,1.2,-2.5,0,0,0,0
5,衣柜,西南角，紧贴墙壁,东西向,-2,-1,0,90,0,0
6,海拉尔的纹章,衣柜门上,,-2.5,-1.5,0,0,0,0
7,服装,衣柜内部,,-2,-1.5,0,0,0,0
8,装备,衣柜内部,,-2,-1.5,0,0,0,0
9,靠背椅,东墙边,东西向,-1,-2,0,90,0,0


#### To Do
1. Look at new incoming industries from the user
2. Match against your data base of values you've already mapped
3. For existing ones, save an API call and get the result from the data base
4. For new ones, batch them together for your LLM to return back to you